<a href="https://colab.research.google.com/github/RoyElkabetz/Text-Summarization-with-Deep-Learning/blob/main/T5_Summarizer_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## uncomment to mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [3]:
!pip install --quiet transformers==4.5.0
!pip install --quiet pytorch-lightning==1.2.7

     |████████████████████████████████| 2.2MB 7.6MB/s 
     |████████████████████████████████| 3.3MB 47.0MB/s 
     |████████████████████████████████| 901kB 40.7MB/s 
     |████████████████████████████████| 839kB 7.6MB/s 
     |████████████████████████████████| 276kB 16.0MB/s 
     |████████████████████████████████| 276kB 26.2MB/s 
     |████████████████████████████████| 829kB 28.0MB/s 
     |████████████████████████████████| 122kB 39.6MB/s 
     |████████████████████████████████| 1.3MB 38.1MB/s 
     |████████████████████████████████| 296kB 40.2MB/s 
     |████████████████████████████████| 143kB 43.3MB/s 


In [4]:
import json
import time
import pandas as pd
import numpy as np
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split
from termcolor import colored
from torchtext.datasets import AG_NEWS, IMDB 

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5TokenizerFast as T5Tokenizer
)

from tqdm.auto import tqdm

pl.seed_everything(216)

Global seed set to 216


216

In [5]:
# plotting packages 
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
rcParams['figure.figsize'] = 16, 10

In [6]:
SAVE_DATASET_PATH = '/content/gdrive/MyDrive/Datasets/Text/IMDB_train_with_summary_dataset.csv'
CHECKPOINTS_PATH = '/content/gdrive/MyDrive/Checkpoints'
MY_MODEL_NAME = 'Text_Summarizer_T5'
MODEL_NAME = 't5-base'
PATH_TO_LAST_CHECKPOINT = ''.join([CHECKPOINTS_PATH, '/', MY_MODEL_NAME, '-v1.ckpt'])

In [7]:
# load the T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

In [8]:
# A class for the model
class NewsSummaryModel(pl.LightningModule):

  def __init__(self):
    super().__init__()
    self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)

  def forward(self, input_ids, attention_mask, decoder_attention_mask, labels=None):

    output = self.model(
        input_ids,
        attention_mask=attention_mask,
        labels=labels,
        decoder_attention_mask=decoder_attention_mask
    )

    return output.loss, output.logits

  def training_step(self, batch, batch_idx):
    input_ids = batch['text_input_ids']
    attention_mask = batch['text_attention_mask']
    labels = batch['labels']
    labels_attention_mask = batch['labels_attention_mask']

    loss, outputs = self(
        input_ids=input_ids,
        attention_mask=attention_mask,
        decoder_attention_mask=labels_attention_mask,
        labels=labels
    )

    self.log('train_loss', loss, prog_bar=True, logger=True)
    return loss
  
  def validation_step(self, batch, batch_idx):
    input_ids = batch['text_input_ids']
    attention_mask = batch['text_attention_mask']
    labels = batch['labels']
    labels_attention_mask = batch['labels_attention_mask']

    loss, outputs = self(
        input_ids=input_ids,
        attention_mask=attention_mask,
        decoder_attention_mask=labels_attention_mask,
        labels=labels
    )

    self.log('valid_loss', loss, prog_bar=True, logger=True)
    return loss

  def test_step(self, batch, batch_idx):
    input_ids = batch['text_input_ids']
    attention_mask = batch['text_attention_mask']
    labels = batch['labels']
    labels_attention_mask = batch['labels_attention_mask']

    loss, outputs = self(
        input_ids=input_ids,
        attention_mask=attention_mask,
        decoder_attention_mask=labels_attention_mask,
        labels=labels
    )

    self.log('test_loss', loss, prog_bar=True, logger=True)
    return loss
    
    
  def configure_optimizers(self):
    return AdamW(self.parameters(), lr=0.0001)

In [9]:
# init model
model = NewsSummaryModel()

In [10]:
checkpoint_callback = ModelCheckpoint(
    dirpath=CHECKPOINTS_PATH,
    filename=MY_MODEL_NAME,
    save_top_k=1,
    verbose=True,
    monitor='valid_loss',
    mode='min'
)

trainer = pl.Trainer(
    checkpoint_callback=checkpoint_callback,
    max_epochs=1,
)

# trainer = pl.Trainer(
#     checkpoint_callback=checkpoint_callback,
#     max_epochs=1,
#     gpus=1,
#     progress_bar_refresh_rate=30
# )

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Checkpoint directory /content/gdrive/MyDrive/Checkpoints exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [11]:
# load best checkpoint of T5 summarizer
trained_model = NewsSummaryModel.load_from_checkpoint(PATH_TO_LAST_CHECKPOINT)
trained_model.freeze()

In [12]:
def summarizer(text):
  # encoding text
  text_encoding = tokenizer(
      text,
      max_length=512,
      padding='max_length',
      truncation=True,
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors='pt'
  )

  # get predictions as ids
  generated_ids = trained_model.model.generate(
      input_ids=text_encoding['input_ids'],
      attention_mask=text_encoding['attention_mask'],
      max_length=150,
      num_beams=2,
      repetition_penalty=2.5,
      length_penalty=1.0,
      early_stopping=True
  )

  # decode and join prediction
  preds = [
   tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
   for gen_id in generated_ids
  ]
  
  return ''.join(preds)

In [ ]:
# # down load dataset as an iterator
# train_iter = IMDB(split='train')
# labels = []
# texts = []

# for i, (label, text) in enumerate(train_iter):
#   labels.append(label)
#   texts.append(text[:512])

# summary = ['empty'] * len(texts)

## create a Pandas DataFrame of data


In [ ]:
# train_df = pd.DataFrame.from_dict({'label': labels, 'text': texts, 'summary': summary})
# train_df = train_df.dropna()
# train_df.head()

,label,text,summary
0,neg,I rented I AM CURIOUS-YELLOW from my video sto...,empty
1,neg,"""I Am Curious: Yellow"" is a risible and preten...",empty
2,neg,If only to avoid making this type of film in t...,empty
3,neg,This film was probably inspired by Godard's Ma...,empty
4,neg,"Oh, brother...after hearing about this ridicul...",empty


In [ ]:
# # check a single example
# with torch.no_grad():
#   sample_row = train_df.iloc[0]
#   text = sample_row['text']
#   model_summary = summarizer(text)

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [ ]:
# text

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to maki'

In [ ]:
# model_summary

"I AM CURIOUS-YELLOW is a film about a young Swedish drama student named Lena who wants to learn everything she can about life. The plot is centered around a maki girl named Maki who wants to focus her attentions on the maki boy's maki, which is a game of maki. The film was released in 1967 and has been rated 4/5 (Sweden)."

## Summarizer Pipeline - get summary and save as a pd.DataFrame

In [13]:
train_df = pd.read_csv(SAVE_DATASET_PATH ,encoding='utf-8')
columns = ['label', 'text', 'summary']
train_df = train_df[columns]
train_df.head()

,label,text,summary
0,neg,I rented I AM CURIOUS-YELLOW from my video sto...,I AM CURIOUS-YELLOW is a film about a young Sw...
1,neg,"""I Am Curious: Yellow"" is a risible and preten...","""I Am Curious: Yellow"" is a risible and preten..."
2,neg,If only to avoid making this type of film in t...,The film is interesting as an experiment but t...
3,neg,This film was probably inspired by Godard's Ma...,Actress Lena Nyman has to be the most annoying...
4,neg,"Oh, brother...after hearing about this ridicul...",After hearing about this ridiculous film for u...


In [ ]:
with torch.no_grad():
  for i in tqdm(range(14131, len(train_df))):
      row = train_df.iloc[i]
      if row['summary'] == 'empty':
        text = row['text']
        summary = summarizer(text)
        train_df.iloc[i, 2] = summary
        if i % 10 == 0:
          train_df.to_csv(path_or_buf=SAVE_DATASET_PATH, columns=columns)

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [34]:
train_df.iloc[14131, 2]

'empty'